In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import h3

from shapely.geometry import Point
from shapely.geometry import Polygon

import contextily as ctx

#from geopy.distance import geodesic
from datetime import datetime


In [ ]:
datetime = "2025-03-19_10-47-56"
filename_DD = f'../data/nextbike/hourly_demand_supply_Dresden {datetime}.csv'
filename_FB = f'../data/nextbike/hourly_demand_supply_Freiburg {datetime}.csv'
df_DD = pd.read_csv(filename_DD, index_col=None, parse_dates=['datetime_hour'])
df_FB = pd.read_csv(filename_FB, index_col=None, parse_dates=['datetime_hour'])

In [ ]:
def plot_df(df, column=None, ax=None, add_basemap=True):
    "Plot based on the `geometry` column of a GeoPandas dataframe"
    df = df.copy()
    df = df.to_crs(epsg=3857)  # web mercator

    if ax is None:
        _, ax = plt.subplots(figsize=(8,8))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    df.plot(
        ax=ax,
        alpha=0.25, edgecolor='k',
        column=column, categorical=True,
        legend=True, legend_kwds={'loc': 'upper left'},
    )
    if add_basemap:
        ctx.add_basemap(ax, crs=df.crs, source=ctx.providers.CartoDB.Positron)

In [ ]:
def plot_shape(shape, ax=None, add_basemap=True):
    df = gpd.GeoDataFrame({'geometry': [shape]}, crs='EPSG:4326')
    plot_df(df, ax=ax, add_basemap=add_basemap)

In [ ]:
def plot_cell(cell, ax=None):
    shape = h3.cells_to_h3shape([cell])
    plot_shape(shape, ax=ax)

In [ ]:
def plot_cells(cells, ax=None):
    fig, ax = plt.subplots(figsize=(8,8))
    shape = h3.cells_to_h3shape(cells)
    plot_shape(shape, ax=ax, add_basemap=True)
    
    for single_cell in cells:
        single_shape = h3.cells_to_h3shape([single_cell])
        # gdf = gpd.GeoDataFrame({'geometry': [single_shape]}, crs='EPSG:4326')
        # gdf = gdf.to_crs(epsg=3857)
        # gdf.plot(ax=ax, alpha=0.5, edgecolor='k')
        plot_shape(single_shape, ax=ax, add_basemap=False)


In [ ]:
df_tmp = df_DD
geometry = [Point(xy) for xy in zip(df_tmp.lng_return, df_tmp.lat_return)]
gdf = gpd.GeoDataFrame(df_tmp, geometry=geometry, crs="EPSG:4326") 

In [ ]:
plot_cells(df_FB['hex_id'].unique())


In [ ]:
plot_cells(df_DD['hex_id'].unique())
